In [ ]:
import math
import os
import sys
# Change according to your path for the hedos package
HEDOS_PATH = '/Users/lmccullum/Documents/hedos'
sys.path.append(HEDOS_PATH)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from blooddvh import BloodDistribution
from blooddvh import tDVH
from blooddvh import bDVH

In [ ]:
# Determine the location of all the input files
INPUT_DIR = os.path.join('input')
OUTPUT_DIR = os.path.join('output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
%%capture
# Perform the analysis for COMPOSITE fields
def get_dvh(INPUT_FIELD_DIR, INPUT_FILE_START, DOSE_PRESCRIPTION, field, N_FRACTIONS):
    # Read in the DVH and convert non-compartments to compartments
    file_path = os.path.join(INPUT_FIELD_DIR, f'{INPUT_FILE_START}_{DOSE_PRESCRIPTION}_F{field}.csv')
    df = pd.read_csv(file_path)
    # Set the initial fractionation and dosing
    dose_data = np.array([float(d) for d in df.iloc[1:,0].to_numpy()])
    dose_per_fraction = (1/N_FRACTIONS) * dose_data
    vol_data = np.array([float(d) for d in df.iloc[1:,1].to_numpy()])
    dvh = np.vstack((dose_per_fraction, vol_data)).transpose()
    return dvh


# For each patient, calculate the bDVH given the liver DVH for multiple fields / dose delivery methods
BP_METHOD = 'weibull' #'markov'
n_BPs = 100000
N_FRACTIONS = 15
INPUT_SUBJECTS = [f for f in os.listdir(INPUT_DIR) if f.startswith('LIV_JP')]
DOSE_PRESCRIPTIONS = ['VMAT', 'IMRT', 'SOBP', 'PBS']

courses_all = pd.DataFrame(index=DOSE_PRESCRIPTIONS, columns=['V0', 'V0_Dmean', 'V0_Dstd',
                                                              'V0.5', 'V0.5_Dmean', 'V0.5_Dstd',
                                                              'V1.0', 'V1.0_Dmean', 'V1.0_Dstd',
                                                              'V2.0', 'V2.0_Dmean', 'V2.0_Dstd',
                                                              'D2%', 'D2%_Dmean', 'D2%_Dstd',
                                                              'D3%', 'D3%_Dmean', 'D3%_Dstd'])
dvh_all = pd.DataFrame(np.empty((n_BPs,len(DOSE_PRESCRIPTIONS)))*np.nan, columns=DOSE_PRESCRIPTIONS)

for SUBJECT in INPUT_SUBJECTS:
    for DOSE_PRESCRIPTION in DOSE_PRESCRIPTIONS:
        print(f'{SUBJECT}, {DOSE_PRESCRIPTION}')

        if SUBJECT.endswith('MALE2'):
            INPUT_FILE_START = 'LIVER, MALE2-LIV-JP-07_TransferredDose'
        else:
            INPUT_FILE_START = 'LIVER-LIV-JP-04_TransferredDose'
        INPUT_FIELD_DIR = os.path.join(INPUT_DIR, SUBJECT)
        INPUT_FIELDS = [f for f in os.listdir(INPUT_FIELD_DIR) if DOSE_PRESCRIPTION in f]

        # Create a beam
        beam = tDVH()

        if DOSE_PRESCRIPTION == 'VMAT':
            for field in range(1, len(INPUT_FIELDS)+1):
                dvh = get_dvh(INPUT_FIELD_DIR, INPUT_FILE_START, DOSE_PRESCRIPTION,
                            field, N_FRACTIONS)
                beam.add_array(60, dvh)
        elif DOSE_PRESCRIPTION == 'IMRT':
            for field in range(1, len(INPUT_FIELDS)+1):
                dvh = get_dvh(INPUT_FIELD_DIR, INPUT_FILE_START, DOSE_PRESCRIPTION,
                            field, N_FRACTIONS)
                beam.add_array(20, dvh)
                beam.add(20, None)
        elif DOSE_PRESCRIPTION == 'SOBP':
            for field in range(1, len(INPUT_FIELDS)+1):
                dvh = get_dvh(INPUT_FIELD_DIR, INPUT_FILE_START, DOSE_PRESCRIPTION,
                            field, N_FRACTIONS)
                beam.add_array(45, dvh)
                beam.add(300, None)
        elif DOSE_PRESCRIPTION == 'PBS':
            field_times = [120, 135, 120]
            for field in range(1, len(INPUT_FIELDS)+1):
                dvh = get_dvh(INPUT_FIELD_DIR, INPUT_FILE_START, DOSE_PRESCRIPTION,
                            field, N_FRACTIONS)
                beam.add_array(field_times[field-1], dvh)
                beam.add(300, None)

        # Create the BP distribution
        BP = BloodDistribution()
        # "_m" for Markov, "_w2" for Weibull
        if BP_METHOD == 'weibull':
            BP.read_from_excel(os.path.join(HEDOS_PATH, INPUT_DIR, 'male.xlsx'), '100k_10min_w2_corrected_volume')
        elif BP_METHOD == 'markov':
            BP.read_from_excel(os.path.join(HEDOS_PATH, INPUT_DIR, 'male.xlsx'), '100k_10min_m')
        # Get the compartment ID
        cid = BP.names.index('liver')
        # Per-DVH
        comp_dose = []
        comp_id = []
        # Append the beam dose and compartment ID to composite
        comp_dose.append(beam)
        comp_id.append(cid)

        # Calculate bDVH per courses and apply course
        try:
            blood_dose = bDVH(BP.df, BP.dt)
            blood_dose.add_dose(comp_dose, comp_id, 0, N_FRACTIONS)
        except AssertionError:
            # Requested treatment time was too short, repeat blood distribution
            n_repeats = math.ceil(comp_dose[0].dvh[0][-1] / BP.df.shape[1])
            blood_dose = bDVH(pd.concat(n_repeats*[BP.df], axis=1, ignore_index=True), BP.dt)
            blood_dose.add_dose(comp_dose, comp_id, 0, N_FRACTIONS)

        # Compile the results to save
        d0 = blood_dose.volume_gt_dose(0)
        d1 = blood_dose.volume_gt_dose(0.5)
        d2 = blood_dose.volume_gt_dose(1.0)
        d3 = blood_dose.volume_gt_dose(2.0)
        v0 = blood_dose.dose_at_top_volume(2.0)
        v1 = blood_dose.dose_at_top_volume(3.0)
        # Save all the results into a dataframe
        courses_all.loc[DOSE_PRESCRIPTION] = [d0[0], d0[1], d0[2],
                                              d1[0], d1[1], d1[2],
                                              d2[0], d2[1], d2[2],
                                              d3[0], d3[1], d3[2],
                                              v0[0], v0[1], v0[2],
                                              v1[0], v1[1], v1[2]]
        dvh_all[DOSE_PRESCRIPTION] = blood_dose.dose_per_fraction[:,0].tolist()

        # Save bDVH figure
        fig,ax = plt.subplots(figsize=(5,5))
        im = ax.imshow(blood_dose.dose_per_fraction, aspect='auto')
        ax.set_xlabel('Fraction #')
        ax.set_ylabel('Blood Particle')
        fig.colorbar(im, orientation='vertical')
        plt.savefig(os.path.join(OUTPUT_DIR, f'fx_img_{BP_METHOD}_{SUBJECT}_{DOSE_PRESCRIPTION}'), dpi=100)
        # Save dose-fractionation map
        blood_dose.dose_per_fraction.tofile(os.path.join(OUTPUT_DIR, f'fx_dmap_{BP_METHOD}_{SUBJECT}_{DOSE_PRESCRIPTION}.bin'))

    # Save the results of all the summary stats and DVHs
    courses_all.to_csv(os.path.join(OUTPUT_DIR, f'fx_summary_all_{BP_METHOD}_{SUBJECT}.csv'), index=True)
    dvh_all.to_csv(os.path.join(OUTPUT_DIR, f'fx_dvhs_all_{BP_METHOD}_{SUBJECT}.csv'), index=True)

In [ ]:
%%capture
# Perform the analysis for PER FIELD
def get_dvh(INPUT_FIELD_DIR, INPUT_FILE_START, DOSE_PRESCRIPTION, field, N_FRACTIONS):
    # Read in the DVH and convert non-compartments to compartments
    file_path = os.path.join(INPUT_FIELD_DIR, f'{INPUT_FILE_START}_{DOSE_PRESCRIPTION}_F{field}.csv')
    df = pd.read_csv(file_path)
    # Set the initial fractionation and dosing
    dose_data = np.array([float(d) for d in df.iloc[1:,0].to_numpy()])
    dose_per_fraction = (1/N_FRACTIONS) * dose_data
    vol_data = np.array([float(d) for d in df.iloc[1:,1].to_numpy()])
    dvh = np.vstack((dose_per_fraction, vol_data)).transpose()
    return dvh


# For each patient, calculate the bDVH given the liver DVH for multiple fields / dose delivery methods
BP_METHOD = 'weibull' # 'markov'
n_BPs = 100000
N_FRACTIONS = 15
INPUT_SUBJECTS = [f for f in os.listdir(INPUT_DIR) if f.startswith('LIV_JP')]
DOSE_PRESCRIPTIONS = ['VMAT', 'IMRT', 'SOBP', 'PBS']

for SUBJECT in INPUT_SUBJECTS:
    for DOSE_PRESCRIPTION in DOSE_PRESCRIPTIONS:
        print(f'{SUBJECT}, {DOSE_PRESCRIPTION}')

        if SUBJECT.endswith('MALE2'):
            INPUT_FILE_START = 'LIVER, MALE2-LIV-JP-07_TransferredDose'
        else:
            INPUT_FILE_START = 'LIVER-LIV-JP-04_TransferredDose'
        INPUT_FIELD_DIR = os.path.join(INPUT_DIR, SUBJECT)
        INPUT_FIELDS = [f for f in os.listdir(INPUT_FIELD_DIR) if DOSE_PRESCRIPTION in f]

        INPUT_INDICES = range(1, len(INPUT_FIELDS)+1)
        courses_all = pd.DataFrame(index=INPUT_INDICES, columns=['V0', 'V0_Dmean', 'V0_Dstd',
                                                                 'V0.5', 'V0.5_Dmean', 'V0.5_Dstd',
                                                                 'V1.0', 'V1.0_Dmean', 'V1.0_Dstd',
                                                                 'V2.0', 'V2.0_Dmean', 'V2.0_Dstd',
                                                                 'D2%', 'D2%_Dmean', 'D2%_Dstd',
                                                                 'D3%', 'D3%_Dmean', 'D3%_Dstd'])
        dvh_all = pd.DataFrame(np.empty((n_BPs,len(INPUT_INDICES)))*np.nan, columns=INPUT_INDICES)

        for field in INPUT_INDICES:
            # Create a beam
            beam = tDVH()
            dvh = get_dvh(INPUT_FIELD_DIR, INPUT_FILE_START, DOSE_PRESCRIPTION, field, N_FRACTIONS)
            if DOSE_PRESCRIPTION == 'VMAT':
                beam.add_array(60, dvh)
            elif DOSE_PRESCRIPTION == 'IMRT':
                beam.add_array(20, dvh)
            elif DOSE_PRESCRIPTION == 'SOBP':
                beam.add_array(45, dvh)
            elif DOSE_PRESCRIPTION == 'PBS':
                field_times = [120, 135, 120]
                beam.add_array(field_times[field-1], dvh)

            # Create the BP distribution
            BP = BloodDistribution()
            # "_m" for Markov, "_w2" for Weibull
            if BP_METHOD == 'weibull':
                BP.read_from_excel(os.path.join(HEDOS_PATH, INPUT_DIR, 'male.xlsx'), '100k_10min_w2_corrected_volume')
            elif BP_METHOD == 'markov':
                BP.read_from_excel(os.path.join(HEDOS_PATH, INPUT_DIR, 'male.xlsx'), '100k_10min_m')
            # Get the compartment ID
            cid = BP.names.index('liver')
            # Per-DVH
            comp_dose = []
            comp_id = []
            # Append the beam dose and compartment ID to composite
            comp_dose.append(beam)
            comp_id.append(cid)

            # Calculate bDVH per courses and apply course
            try:
                blood_dose = bDVH(BP.df, BP.dt)
                blood_dose.add_dose(comp_dose, comp_id, 0, N_FRACTIONS)
            except AssertionError:
                # Requested treatment time was too short, repeat blood distribution
                n_repeats = math.ceil(comp_dose[0].dvh[0][-1] / BP.df.shape[1])
                blood_dose = bDVH(pd.concat(n_repeats*[BP.df], axis=1, ignore_index=True), BP.dt)
                blood_dose.add_dose(comp_dose, comp_id, 0, N_FRACTIONS)

            # Compile the results to save
            d0 = blood_dose.volume_gt_dose(0)
            d1 = blood_dose.volume_gt_dose(0.5)
            d2 = blood_dose.volume_gt_dose(1.0)
            d3 = blood_dose.volume_gt_dose(2.0)
            v0 = blood_dose.dose_at_top_volume(2.0)
            v1 = blood_dose.dose_at_top_volume(3.0)
            # Save all the results into a dataframe
            courses_all.loc[field] = [d0[0], d0[1], d0[2],
                                      d1[0], d1[1], d1[2],
                                      d2[0], d2[1], d2[2],
                                      d3[0], d3[1], d3[2],
                                      v0[0], v0[1], v0[2],
                                      v1[0], v1[1], v1[2]]
            dvh_all[field] = blood_dose.dose_per_fraction[:,0].tolist()

            # Save bDVH figure
            fig,ax = plt.subplots(figsize=(5,5))
            im = ax.imshow(blood_dose.dose_per_fraction, aspect='auto')
            ax.set_xlabel('Fraction #')
            ax.set_ylabel('Blood Particle')
            fig.colorbar(im, orientation='vertical')
            plt.savefig(os.path.join(OUTPUT_DIR, f'fx_img_{BP_METHOD}_{SUBJECT}_{DOSE_PRESCRIPTION}_F{field}'), dpi=100)
            # Save dose-fractionation map
            blood_dose.dose_per_fraction.tofile(os.path.join(OUTPUT_DIR, f'fx_dmap_{BP_METHOD}_{SUBJECT}_{DOSE_PRESCRIPTION}_F{field}.bin'))

        # Save the results of all the summary stats and DVHs
        courses_all.to_csv(os.path.join(OUTPUT_DIR, f'fx_summary_all_{BP_METHOD}_{SUBJECT}_{DOSE_PRESCRIPTION}.csv'), index=True)
        dvh_all.to_csv(os.path.join(OUTPUT_DIR, f'fx_dvhs_all_{BP_METHOD}_{SUBJECT}_{DOSE_PRESCRIPTION}.csv'), index=True)